Wir verwenden das Modul gensim um ein Word2Vec Modell zu trainieren

In [1]:
# !pip install gensim 

In [1]:
from gensim.models import Word2Vec
import pickle
import math
import numpy  as np
import pandas as pd
import re

In [2]:
import nltk
import json

Über diese Funtkion können die Umgebungen aus dem Skipgram vorhergesagt werden

In [3]:
from numpy import exp, dot
from gensim import matutils
#https://github.com/RaRe-Technologies/gensim/issues/2152
def predict_output_context(model, center_word, topn=10):
    word = model.wv.vocab[center_word]
    vec = model.wv.vectors[word.index]
    prob_values = exp(dot(vec, model.trainables.syn1neg.T))
    prob_values /= sum(prob_values)
    top_indices = matutils.argsort(prob_values, topn=topn, reverse=True)
    return [(model.wv.index2word[index1], prob_values[index1]) for index1 in top_indices]

Wir haben Modelle mit und ohne Stopwords erzeugt. 

Laden der Stopwords von nltk und der aus den Daten erzeugten Stopwords.

In [3]:
from nltk.corpus import stopwords
nltkStopWords = stopwords.words("german")

In [4]:
with open("all_stop_words.json", "r") as f:
    createdStopWords = json.load(f)

Diese Funktion nimmt eine Liste als Parameter und generiert daraus eine RegEx die jeden String der Liste matcht 

In [4]:
def buildRegexFromList(stringsToMatch):
    init = ""
    for s in stringsToMatch:
        init += r'\b'
        init += re.escape(s)
        init += r'\b|'
    #remove last pipe:
    init = init[:-1]
    return init

Regex um beispielsweise Paragraphenzeichen zu entfernen

In [15]:
regexNoSpecialStuff = r"[^A-zäöüÄOÜß\s]"

Erzeugen einer Liste aller Stopwords 

In [14]:
additional_custom_stopwords = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ',,', '[', ']', 'vgl', '...', \
                               'ovg', 'nrw','de',"a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "bzw", \
                               "k", "l", "m", "n", "o", "p", "q", "r","s", "t", "u", "v", "w", "x", "y", "z"]


for newStop in additional_custom_stopwords:
    if newStop not in nltkStopWords:
        nltkStopWords.append(newStop)
        
for newStop in createdStopWords:
    if newStop not in nltkStopWords:
        nltkStopWords.append(newStop)

NameError: name 'nltkStopWords' is not defined

Daraus die RegEx generieren

In [5]:
regExStopWords = buildRegexFromList(nltkStopWords)

NameError: name 'nltkStopWords' is not defined

Alle Sätze laden

In [7]:
with open("../allSentences.pkl", "rb") as f:
    sentences = pickle.load(f)

1/4  der Daten wird zufällig behalten, der Rest gelöscht

In [8]:
retain = math.floor(len(sentences)/4)
smallerSents = np.random.choice(sentences, retain)

In [9]:
del sentences

In [14]:
del smallerSents

In [16]:
concated = []

Sätze lowern und mit Regex bereinigen, hier können Stopwords entfernt weden oder auch nicht

In [17]:
def cleanSentence(sent):
    sent = sent.lower()
#    sent = re.sub(regExStopWords, "", sent)
    sent = re.sub(regexNoSpecialStuff, "", sent)
    return sent

In [18]:
len(smallerSents)

26160

Da die Sätze in einem nested Array sind, wobei jeder Eintrag in der ersten Ebene einem Dokument entspricht, iterieren wir über die Sätze und wenden sowohl die cleanSentence Funktion als auch word_tokenize von nltk an, um alle Sätze zu cleanen und tokenizieren

In [19]:
for i in range(len(smallerSents)):
    concated.append([nltk.word_tokenize(cleanSentence(s)) for s in smallerSents[i]])
    if (i%1000 ==0):
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000


In [20]:
del smallerSents

In [21]:
len(concated)

26160

In [15]:
nltk.word_tokenize("Test Satz", language="german")


['Test', 'Satz']

Wir nehmen nur Sätze, die mehr als 3 Worte enthalten

In [22]:
allTokenized = []
for idx, part in enumerate(concated):
    for sent in part:
        if(len(sent)>3):
            allTokenized.append(sent)
            
    if (idx%100 == 0):
        print(idx)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

Damit wird jetzt ein word2vec Model trainiert. Dabei werden nur Wörter die mindestens 5 mal Vorkommen berücksichtigt. Der Parameter sg=1 aktiviert SkipGram statt CBOW

In [23]:
model = Word2Vec(allTokenized, min_count=5, sg=1)

In [24]:
print(model)

Word2Vec(vocab=198337, size=100, alpha=0.025)


In [25]:
print(list(model.wv.vocab)[:10])

['beschwerde', 'antragsteller', 'beschluss', 'sozialgerichts', 'duisburg', 'geändert', 'antragsgegnerin', 'einstweilig', 'verpflichtet', 'antragstellern']


Speichern der Modelle

In [25]:
model.save("withStopwords_w2v.model")

Hier einige Test-Ausgaben

In [32]:
model.wv.most_similar("fettdruck")

[('kursiv', 0.7948729991912842),
 ('schriftgröße', 0.7640509605407715),
 ('drucktechnisch', 0.7635499238967896),
 ('sperrschrift', 0.740163266658783),
 ('überschriften', 0.7324076890945435),
 ('text', 0.7287232875823975),
 ('fließtext', 0.7165791988372803),
 ('farblich', 0.7145205736160278),
 ('kleingedruckten', 0.7116425037384033),
 ('widerrufsfolgen', 0.7100821733474731)]

In [38]:
predict_output_context(model=model, center_word="gefängnis".lower(), topn=8)

[('gefängnis', 0.020155497),
 ('gefoltert', 0.003620692),
 ('festgenommen', 0.0031493008),
 ('bestraft', 0.0030065014),
 ('geflohen', 0.0028015247),
 ('batman', 0.002466482),
 ('geflüchtet', 0.0022763843),
 ('zurückgekehrt', 0.0021883382)]

In [27]:
model.corpus_count

3207576

In [28]:
model.wv.get_vector("antrag")

array([ 0.06819087, -0.05849149, -0.44034144,  0.43365505,  0.35931936,
        0.23948576,  0.21274006, -0.11572772, -0.09004484,  0.23248108,
        0.1180741 , -0.4541765 , -0.19107921, -0.22647363,  0.24266577,
       -0.11351085,  0.13632649,  0.09431722,  0.1741414 , -0.2821583 ,
       -0.20260008, -0.24789643, -0.06995609, -0.33414626, -0.72408265,
       -0.17213352, -0.45253775,  0.3872577 ,  0.46451366,  0.247034  ,
       -0.22019464, -0.5967129 ,  0.28587347,  0.26321846,  0.24335924,
        0.52247864,  0.12950346,  0.7205421 , -0.27208588,  0.3231255 ,
        0.23748551, -0.04112607, -0.6253382 , -0.252556  , -0.08153218,
        0.7452898 ,  0.02241766, -0.5604143 ,  0.4858182 , -0.18336153,
        0.49712628,  0.16229424,  0.02963648,  0.23470663, -0.6413679 ,
        0.07037396,  0.03973873,  0.44359607, -0.43676433, -0.017108  ,
       -0.05136756, -0.48984584,  0.10393927, -0.5248685 ,  0.5380487 ,
       -0.4540248 , -0.25822267, -0.07911491, -0.12509339,  0.20

In [29]:
model.vocabulary.sorted_vocab

True